In [1]:
# Pandas for DataFrames
import pandas as pd
pd.set_option('display.max_column', 100)

# NumPy for numerical computing
import numpy as np
np.random.seed(123)
import random
random.seed(123)

import os

# Matplotlib for visualization
from matplotlib import pyplot as plt

# display plots in the notebook
%matplotlib inline

In [2]:
import sys
sys.path.append('./utils')

from data import Data
from models import Models
from tags import Tags
tags = Tags()

Using TensorFlow backend.


In [3]:
PLANET_KAGGLE_ROOT = '/data/planet-data/'
if not os.path.exists(PLANET_KAGGLE_ROOT):
    PLANET_KAGGLE_ROOT = '/Users/jiayou/Documents/Kaggle Data/Amazon'
    
N_TAGS = 17
N_TEST_T = 40669
N_TEST_F = 20522
N_TEST = N_TEST_T + N_TEST_F

In [5]:
thres = [0.23067564, 0.27402788, 0.15499838, 0.18645976, 0.12418672, 0.093219191, 0.14909597, 0.13256209, 0.041971382, 0.17731731, 0.10376091, 0.25468382, 0.090709485, 0.13336645, 0.13344041, 0.10004906, 0.036582272]
thres = [0.2]*17

def pred_to_output(pred):
    result = pd.DataFrame({
        'image_name': 
            ['test_{}'.format(i) for i in range(N_TEST_T)] + ['file_{}'.format(i) for i in range(N_TEST_F)],
        'tags': ['' for i in range(N_TEST)]
    })
    for i in range(len(pred)):
        current_pred = pred[i]
        current_tag = tags.pred_to_tags(current_pred, thres=thres)
        result.iat[i, 1] = current_tag
    return result

def consolidate(pred):
    return pred.mean(axis=0)

pred = None
pred8 = None

def predict(model_path, toy=None, batch_size=20):
    model = Models.load_resnet50(model_path)
    print('Model weights loaded')
    
    d = Data(train=[], toy=toy)
    
    cnt = 0
    global pred
    global pred8
    pred = np.zeros((N_TEST, N_TAGS))
    pred8 = np.zeros((N_TEST * 8, N_TAGS))
    print('Start predicting..')
    for X_test in d.gen_test_augmented(batch_size):
        y_test = model.predict_on_batch(X_test)
        k = int(len(y_test) / 8 + 0.1)
        pred8[cnt*8:(cnt+k)*8,:] = y_test[:,:]
        for i in range(k):
            pred[cnt+i,:] = consolidate(y_test[8*i:8*(i+1),:])
        cnt += k
        print('Predicted {} images'.format(cnt))
    print('Predicted all {} images'.format(cnt))
            
    print('Saving raw predictions...')
    np.save('raw_pred.v9.tta.npy', pred)
    np.save('raw_pred.v9.tta8.npy', pred8)
    print('Saved')
    
    result = pred_to_output(pred)
    print('Saving submission file...')
    result.to_csv('submission.v9.tta.csv', index = None)
    print('Saved')
    return result

In [11]:
model = Models.load_resnet50('weights-v9.hdf5')

In [19]:
model.layers[2].get_weights()[0][0,0,0]

array([ 0.00223983, -0.01109277,  0.02911913, -0.01754992,  0.00387825,
       -0.04665675,  0.03481624,  0.00356703, -0.04247806,  0.03060204,
       -0.01988623, -0.00865006, -0.04124454, -0.01158302,  0.01692761,
       -0.0108131 ,  0.02604449, -0.03988459, -0.02094269, -0.01312441,
       -0.03182349,  0.0339497 ,  0.00557818,  0.01078453, -0.00401395,
       -0.04496027,  0.0263937 ,  0.00038293,  0.01641451,  0.00056055,
       -0.02893478,  0.03983829,  0.0289033 ,  0.0401258 ,  0.01868348,
       -0.0391048 , -0.02870469,  0.0012854 ,  0.01966462, -0.01625841,
        0.0053267 , -0.00713183,  0.00480656, -0.0394246 , -0.02244909,
       -0.03423842,  0.02129313, -0.02206269,  0.0393597 ,  0.03589353,
       -0.02190847, -0.00146624,  0.05018585, -0.03080506, -0.04157066,
       -0.00952809,  0.02269129, -0.03925563,  0.00944098, -0.01765026,
       -0.02366939, -0.0193019 , -0.02678562, -0.0240464 ], dtype=float32)

In [6]:
out = predict('./weights-v9.hdf5')

Model weights loaded
Loading data...
Loading done
Start predicting..
Predicted 5 images
Predicted all 5 images
Saving raw predictions...
Saved
Saving submission file...
Saved


In [ ]:
pred = np.load('raw_pred.v9.npy')
result = pred_to_output(pred)
result.to_csv('submission.v9-2.csv', index = None)